In [ ]:
"""
activity i need to capture:

- total commits
- total lines of code added
- total lines of code removed

- commits per repository 
    - display org and reponame together with number of commits
- repositories created
    - name of repo
- issues opened 
    - name of repo and headline of issue
- pull requests opened
    - name of repo and title of pull request


# how it works

- github cronjob daily at the end of my day 00:00 - collects data until back 00:01
- telegram bot sends me a message with the data formatted
"""

In [30]:
import requests
from datetime import datetime, timedelta
import pytz
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def fetch_last_event_local_time(username, token):
    if not username or not token:
        raise ValueError("GitHub username or token is not set.")
    
    headers = {'Authorization': f'token {token}'}
    url = f'https://api.github.com/users/{username}/events'
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        raise Exception(f"Error fetching events: {response.status_code}")

    events = response.json()
    if not events:
        print("No events found for the user.")
        return None
    
    last_event_time_utc = events[0]['created_at']  # The last event's timestamp

    # Convert to local time
    dt = datetime.strptime(last_event_time_utc, '%Y-%m-%dT%H:%M:%SZ')
    utc_time = dt.replace(tzinfo=pytz.UTC)
    local_time = utc_time.astimezone()  # Convert to local timezone

    return local_time

In [31]:
def get_time_24_hours_back(event_time):
    # Get the current time in the same timezone as the event time
    current_time = datetime.now(event_time.tzinfo)
    
    # Calculate the time 24 hours back
    time_24_hours_back = current_time - timedelta(hours=24)
    
    return time_24_hours_back

# Example usage
username = os.getenv('GITHUB_USER')
token = os.getenv('GITHUB_TOKEN')
event_time = fetch_last_event_local_time(username, token)
if event_time:
    time_24_hours_back = get_time_24_hours_back(event_time)
    print("Current time:", datetime.now(event_time.tzinfo))
    print("Time 24 hours back:", time_24_hours_back)

Current time: 2024-09-07 13:09:56.142089+02:00
Time 24 hours back: 2024-09-06 13:09:56.142084+02:00


In [35]:
import requests
from datetime import datetime, timedelta
import pytz
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def fetch_last_event_local_time(username, token):
    if not username or not token:
        raise ValueError("GitHub username or token is not set.")
    
    headers = {'Authorization': f'token {token}'}
    url = f'https://api.github.com/users/{username}/events'
    response = requests.get(url, headers=headers)
    
    if response.status_code == 401:
        raise Exception("Unauthorized: Check your GitHub token.")
    elif response.status_code != 200:
        raise Exception(f"Error fetching events: {response.status_code}")

    events = response.json()
    if not events:
        print("No events found for the user.")
        return None
    
    last_event_time_utc = events[0]['created_at']  # The last event's timestamp

    # Convert to local time
    dt = datetime.strptime(last_event_time_utc, '%Y-%m-%dT%H:%M:%SZ')
    utc_time = dt.replace(tzinfo=pytz.UTC)
    local_time = utc_time.astimezone()  # Convert to local timezone

    return local_time

def get_time_24_hours_back(event_time):
    # Get the current time in the same timezone as the event time
    current_time = datetime.now(event_time.tzinfo)
    
    # Calculate the time 24 hours back
    time_24_hours_back = current_time - timedelta(hours=24)
    
    return time_24_hours_back

def fetch_repositories(username, token):
    headers = {'Authorization': f'token {token}'}
    url = f'https://api.github.com/users/{username}/repos'
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        raise Exception(f"Error fetching repositories: {response.status_code}")
    
    return response.json()

def fetch_commits(username, repo, token, since, until):
    headers = {'Authorization': f'token {token}'}
    url = f'https://api.github.com/repos/{username}/{repo}/commits'
    params = {'since': since.isoformat(), 'until': until.isoformat()}
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code != 200:
        raise Exception(f"Error fetching commits for {repo}: {response.status_code}")
    
    return response.json()

def count_total_commits(username, token, since, until):
    repos = fetch_repositories(username, token)
    total_commits = 0
    
    for repo in repos:
        repo_name = repo['name']
        try:
            commits = fetch_commits(username, repo_name, token, since, until)
            total_commits += len(commits)
        except Exception as e:
            print(f"Skipping repository {repo_name} due to error: {e}")
    
    return total_commits

# Example usage
username = os.getenv('GITHUB_USER')
token = os.getenv('GITHUB_TOKEN')
event_time = fetch_last_event_local_time(username, token)
if event_time:
    time_24_hours_back = get_time_24_hours_back(event_time)
    total_commits = count_total_commits(username, token, time_24_hours_back, event_time)
    print("Total commits in the last 24 hours:", total_commits)

Skipping repository Attention_is_all_you_need due to error: Error fetching commits for Attention_is_all_you_need: 409
Total commits in the last 24 hours: 2
